## Sharding of LLMs up to 7B

The free environment of colab without a GPU has a large amount of RAM (around 25GB), which means you can use it to load models with large weights that may not fit in an environment with a GPU.

Sharding allows the use of Hugging Face's "Accelerate" for loading models, which reduces RAM requirements.

Forked from a colab doc of: https://huggingface.co/CleverShovel; Original link: https://colab.research.google.com/drive/12sqdlzv9vRuYhDFbh0kIU8BL7de24uST?usp=sharing

Adapted by: RonanKMcGovern of https://research.trelis.com and https://huggingface.co/Trelis

### Install requirements

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q einops
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00


### Login to Huggingface Hub

You need to create token with write role [here](https://huggingface.co/settings/tokens) and input in cell below.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Model loading

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model_name = 'meta-llama/Llama-2-7b-chat-hf'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    offload_folder='offload'
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Saving to the hub

In [ ]:
save_model = f"{model_name.split('/')[1]}-sharded-bf16-5GB"

model.push_to_hub(save_model, max_shard_size="5GB")
tokenizer.push_to_hub(save_model)

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RonanMcGovern/Llama-2-7b-chat-hf-sharded-bf16-5GB/commit/8ba2328370bea41440236d3c625ff1b0ff8f6561', commit_message='Upload tokenizer', commit_description='', oid='8ba2328370bea41440236d3c625ff1b0ff8f6561', pr_url=None, pr_revision=None, pr_num=None)